In [10]:
import numpy as np
import random

from env.Board import Board
from env.GameController import GameController
from env.GameState import GameState
from env.Player import Player
from env.RuleEngine import RuleEngine

class GameController:
    """
    Manages the overall flow of an Oware game, controlling game rounds, player actions, and the game state.

    This controller sets up the game, decides the starting player, manages the sequence of turns within each round,
    and checks for end-of-game conditions. It interacts with the `Board`, `Player`, and `GameState` classes to
    execute the game logic and maintain the state of the game.

    Attributes:
        n_players (int): Number of players in the game, typically two.
        board (Board): An instance of the Board class representing the game board.
        player (Player): An instance of the Player class to manage player actions.
        environment (GameState): An instance of the GameState class to track and store the game's state.
        rules (RuleEngine): An instance of the RuleEngine class to enforce game rules.
        max_turns (int): The maximum number of turns allowed to prevent infinite loops.

    Methods:
        __init__(num_of_rounds=7): Initializes the game controller with a specified number of rounds.
        starting_player(how="random"): Determines which player starts a round.
        choose_action_player(player): Randomly selects a valid action for the given player.
        game(): Executes the main game loop, handling the progression of rounds and managing game state updates.

    The game() method is the core function that runs the game loop, orchestrating the game by managing rounds,
    processing player actions, updating the state, and determining when the game ends based on the rules defined
    in the RuleEngine.

    Example:
        num_of_rounds = 7
        game = GameController(num_of_rounds)
        game.game()  # Start the game loop
    """

    def __init__(self):
        self.n_players = 2
        self.board = Board()
        self.environment = GameState(self.board)
        self.player = Player(self.board, self.environment)
        self.rules = RuleEngine(self.board, self.environment)
        self.max_turns = 2000

    def starting_player(self, how="random"):
        """
        Determines the starting player of a round based on the specified method.

        Parameters:
            how (str): Method to determine the starting player. Options are "random" or "last_winner".

        Returns:
            int: The player number who will start the round.
        """
        if how == "random":
            starter = random.sample([1, 2], 1)
            return starter[0]
        elif how == "last_winner" and len(self.environment.win_list)!=0:
            starter = self.environment.win_list[-1]
            return starter
        else:
            starter = 1
            return starter


    def choose_action_player(self, player):
        """
        Selects a valid action for the player randomly from the possible moves.

        Parameters:
            player (int): The player number for whom to select the action.

        Returns:
            int: The action index chosen for the player.
        """
        return random.sample(self.environment.possible_moves(player), 1)[0]



    def game(self, num_of_rounds):
        print(f"START GAME ...")

        """
        Executes the game loop, managing rounds and player turns until game completion conditions are met.

        This loop orchestrates the game flow by:
        1. Determining which player starts each round, alternating starting players based on random choice for the first round and the winner of the previous round for subsequent rounds.
        2. Incrementing the round count and processing individual turns within each round until there are no seeds left on the board or a stopping condition is triggered.
        3. Each player's turn involves choosing a valid action (pit from which to distribute seeds), distributing seeds from the selected pit, and potentially capturing seeds from the board.
        4. After each action, the game state is saved to record the actions and board state.
        5. The loop checks if the round should stop (based on the board state or other game rules), and if so, breaks out of the round loop to start a new round.
        6. After the completion of each round, the loop checks for game end conditions such as reaching a maximum number of rounds or a specific condition that defines the end of the game.
        7. Updates the game statistics, including the number of games won by each player and updates to territory counts based on the results of the round.
        8. Resets the board to its initial state at the end of each round and prepares for the next round if the game has not reached its conclusion.

        Parameters:
        - num_of_rounds (int): Maximum number of rounds to be played.
        - self.max_turns (int): Maximum number of turns allowed, to prevent potentially infinite games.

        Outputs:
        - Prints the current round and turn, the state of the board after each turn, and messages at the end of each round and game.
        - Updates internal state to track rounds, player scores, and other game metrics.

        Side effects:
        - Modifies the internal state of the board, players, and game controller to reflect the progression of the game.
        """

        
        while self.rules.round < num_of_rounds and self.board.turns_completed < self.max_turns:

            print(f" START ROUND {self.rules.round}")
            print("\n")
            
            # Decision on who starts the round
            if self.rules.round == 1:
                current_player = self.starting_player("random")
                other_player = 1 if current_player == 2 else 2
            else:
                current_player = self.starting_player("last_winner")
                other_player = 1 if current_player == 2 else 2
            print(f"Player {current_player} starts this round")

            # increment the number of rounds after the choice of who starts the round
            self.rules.round +=1
            
            # Implement a round
            while np.sum(self.board.board, axis = None) > 0:
                self.environment.save_all_states()
                self.environment.save_game_state()
                action_c = self.choose_action_player(current_player)
                print(f"Current player: {current_player}")
                print(f"Player {current_player} action options {self.environment.possible_moves(current_player)}\n")

                print(f"Player {current_player} chooses action: {action_c}\n")

                self.player.player_step(action_c, current_player, other_player)
                self.environment.save_actions(current_player, action_c)
                print("GAME STATE SAVING ...")
                self.environment.save_all_states()
                self.environment.save_game_state()
                print("GAME STATE SAVED ...")
                print(f"Total states saved ({len(self.environment.game_state)})\n")

                print(f"Switch Players ...")
                self.environment.switch_player(current_player, other_player)
                
                if self.rules.stop_round() == True:             
                    break
                
                action_o = self.choose_action_player(other_player)

                print(f"Current player: {other_player}")
                print(f"Player {other_player} action options {self.environment.possible_moves(other_player)}\n")

                print(f"Player {other_player} chooses action: {action_o}\n")
                self.player.player_step(action_o,  other_player, current_player)
                self.environment.save_actions(other_player, action_o)
                print("GAME STATE SAVING ...")
                self.environment.save_game_state()
                self.environment.save_all_states()
                print("GAME STATE SAVED ...")
                
                # if self.board.turns_completed == 2000:
                #     print("2000 turns in round reached")
                #     break

            # if self.board.turns_completed == 2000:
            #     break

            if self.board.stores[0] > self.board.stores[1]:
                self.environment.games_won[0] +=1
                self.environment.win_list += [1]
                self.board.territory_count[0] +=1
                self.board.territory_count[1] -=1
                # write function to update territories

            elif self.board.stores[0] < self.board.stores[1]:
                self.environment.games_won[1] +=1
                self.environment.win_list += [2]
                self.board.territory_count[1] +=1
                self.board.territory_count[0] -=1

            self.environment.rounds_completed += 1
            winner = self.environment.game_winner()
            self.environment.save_winner() 

            print(f"ROUND ENDS\n")
            print(f"Rounds completed: {self.environment.rounds_completed }")

            print('RESET BOARD AND STORES FOR NEW ROUND ...')
            self.board.reset_board()
            self.board.stores = np.array([0, 0])

            if self.rules.stop_game() == True:
                print(f"STOP GAME")
                break
            
            if winner != 0:
                print(f"Player {winner} wins!")
            else: 
                print(f" Game ends in draw ")

In [7]:
board_states = []
sub_states = []
fresh_board = np.zeros((2, 6))
board_states.append(fresh_board)
actions = list(range(12))
for i in actions:
    for element in sub_states:
        new_board = Player().player_step(element)
        sub_states.append(new_board)


In [8]:
sub_states

[]

In [22]:
import numpy as np
import random
from copy import deepcopy

from env.Board import Board
from env.GameState import GameState
from env.Player import Player
from env.RuleEngine import RuleEngine

class GameSimulator:
    def __init__(self):
        self.n_players = 2
        self.board = Board()  # Assuming you've implemented the Board class
        self.environment = GameState(self.board)  # Assuming you've implemented the GameState class
        self.player = Player(self.board, self.environment)  # Assuming you've implemented the Player class
        self.rules = RuleEngine(self.board, self.environment)  # Assuming you've implemented the RuleEngine class
        self.max_turns = 2000
        self.board_states = []
        self.sub_states = [deepcopy(self.board.board)]

    def find_next_states(self, current_board):
        next_states = []
        for action in range(12):  # Assuming there are 12 possible actions
            pit_index = self.board.action2pit(action)
            if current_board[pit_index] != 0:
                copied_board = deepcopy(current_board)
                self.player.player_step(action, 1, 2)  # Assuming player_step modifies the board
                next_states.append(deepcopy(self.board.board))
        return next_states

    def game(self):
        print("START GAME ...")
        turn = 0
        while turn < self.max_turns and any(np.any(seeds != 0) for seeds in self.sub_states):
            next_level_states = []
            print(f"Turn {turn}:")
            for state in self.sub_states:
                print(f"Current board:\n{state}")
                next_states = self.find_next_states(state)
                print(f"Next states:\n{next_states}")
                self.board_states.extend(next_states)
                next_level_states.extend(next_states)
            self.sub_states = next_level_states
            turn += 1
            if not self.sub_states:
                print("No more moves possible.")
                break

        print("Simulation completed.")

# Usage example
if __name__ == "__main__":
    simulator = GameSimulator()
    simulator.game()


START GAME ...
Turn 0:
Current board:
[[4 4 4 4 4 4]
 [4 4 4 4 4 4]]
Next states:
[array([[6, 1, 6, 1, 7, 2],
       [6, 6, 0, 1, 6, 6]]), array([[7, 2, 7, 2, 0, 2],
       [7, 7, 1, 1, 6, 6]]), array([[9, 1, 0, 1, 1, 0],
       [9, 1, 3, 3, 8, 8]]), array([[9, 1, 0, 1, 1, 0],
       [9, 1, 3, 3, 8, 8]]), array([[ 0,  0,  1,  2,  2,  1],
       [10,  2,  4,  4,  9,  9]]), array([[ 0,  0,  1,  2,  2,  1],
       [10,  2,  4,  4,  9,  9]]), array([[ 0,  1,  2,  3,  3,  2],
       [ 0,  3,  5,  5, 10, 10]]), array([[ 2,  0,  4,  5,  5,  4],
       [ 2,  0,  8,  1,  1, 12]]), array([[ 2,  1,  5,  6,  6,  5],
       [ 2,  0,  0,  2,  2, 13]]), array([[4, 3, 7, 8, 1, 8],
       [4, 2, 2, 0, 5, 0]]), array([[ 6,  0,  1, 10,  0, 10],
       [ 6,  4,  4,  0,  2,  1]]), array([[ 7,  1,  2, 11,  1,  0],
       [ 7,  5,  5,  1,  3,  1]])]
Turn 1:
Current board:
[[6 1 6 1 7 2]
 [6 6 0 1 6 6]]
Next states:
[array([[ 7,  1,  2, 11,  1,  0],
       [ 7,  5,  5,  1,  3,  1]]), array([[8, 2, 3, 1, 1, 1]

In [24]:
def find_unique_boards(board_states):
    unique_boards = set()
    # Convert each board array to a tuple and add to the set for uniqueness
    for board in board_states:
        board_tuple = tuple(map(tuple, board))  # Convert array to tuple of tuples
        unique_boards.add(board_tuple)
    
    # Convert tuples back to arrays if you need them in array form later
    unique_board_arrays = [np.array(board) for board in unique_boards]
    
    return unique_board_arrays


unique_boards = find_unique_boards(simulator.board_states)



In [26]:
unique_boards, len(unique_boards)

([array([[0, 2, 1, 0, 0, 0],
         [3, 1, 1, 0, 0, 0]]),
  array([[5, 0, 2, 3, 0, 0],
         [1, 1, 0, 0, 0, 0]]),
  array([[3, 3, 6, 2, 7, 7],
         [1, 1, 0, 1, 1, 8]]),
  array([[1, 2, 1, 1, 0, 2],
         [0, 0, 3, 1, 0, 1]]),
  array([[4, 2, 0, 1, 2, 3],
         [0, 0, 0, 0, 0, 0]]),
  array([[1, 2, 1, 1, 1, 0],
         [0, 0, 0, 0, 0, 2]]),
  array([[ 6,  0,  1, 10,  0, 10],
         [ 6,  4,  4,  0,  2,  1]]),
  array([[4, 2, 1, 2, 3, 0],
         [0, 0, 0, 0, 0, 0]]),
  array([[4, 2, 0, 0, 0, 0],
         [0, 3, 2, 1, 0, 0]]),
  array([[0, 0, 1, 1, 1, 3],
         [8, 6, 3, 6, 3, 4]]),
  array([[0, 1, 2, 1, 3, 5],
         [0, 0, 5, 8, 5, 6]]),
  array([[9, 1, 0, 0, 0, 2],
         [7, 5, 2, 5, 2, 3]]),
  array([[2, 0, 0, 1, 1, 2],
         [2, 0, 0, 0, 0, 0]]),
  array([[1, 1, 2, 0, 2, 5],
         [0, 0, 0, 0, 0, 1]]),
  array([[4, 2, 0, 0, 1, 2],
         [0, 0, 0, 0, 0, 3]]),
  array([[1, 4, 5, 4, 3, 9],
         [3, 2, 1, 0, 0, 0]]),
  array([[4, 2, 0, 0, 0, 0],

In [27]:
simulator.board_states, len(simulator.board_states)

([array([[6, 1, 6, 1, 7, 2],
         [6, 6, 0, 1, 6, 6]]),
  array([[7, 2, 7, 2, 0, 2],
         [7, 7, 1, 1, 6, 6]]),
  array([[9, 1, 0, 1, 1, 0],
         [9, 1, 3, 3, 8, 8]]),
  array([[9, 1, 0, 1, 1, 0],
         [9, 1, 3, 3, 8, 8]]),
  array([[ 0,  0,  1,  2,  2,  1],
         [10,  2,  4,  4,  9,  9]]),
  array([[ 0,  0,  1,  2,  2,  1],
         [10,  2,  4,  4,  9,  9]]),
  array([[ 0,  1,  2,  3,  3,  2],
         [ 0,  3,  5,  5, 10, 10]]),
  array([[ 2,  0,  4,  5,  5,  4],
         [ 2,  0,  8,  1,  1, 12]]),
  array([[ 2,  1,  5,  6,  6,  5],
         [ 2,  0,  0,  2,  2, 13]]),
  array([[4, 3, 7, 8, 1, 8],
         [4, 2, 2, 0, 5, 0]]),
  array([[ 6,  0,  1, 10,  0, 10],
         [ 6,  4,  4,  0,  2,  1]]),
  array([[ 7,  1,  2, 11,  1,  0],
         [ 7,  5,  5,  1,  3,  1]]),
  array([[ 7,  1,  2, 11,  1,  0],
         [ 7,  5,  5,  1,  3,  1]]),
  array([[8, 2, 3, 1, 1, 1],
         [8, 6, 6, 2, 4, 2]]),
  array([[8, 2, 0, 0, 1, 1],
         [8, 6, 6, 2, 4, 2]]),
  ar

In [ ]:
new_board = self.board.board
g = GameSimulator()
g.sub_states_states.append(new_board)

for action in range(12):
    


In [38]:
class GameSimulator:
    def __init__(self):
        self.n_players = 2
        self.board = Board()  # Assuming you've implemented the Board class
        self.environment = GameState(self.board)  # Assuming you've implemented the GameState class
        self.player = Player(self.board, self.environment)  # Assuming you've implemented the Player class
        self.rules = RuleEngine(self.board, self.environment)  # Assuming you've implemented the RuleEngine class
        self.max_turns = 2000
        self.board_states = []  # Store all encountered board states
        self.sub_states = [deepcopy(self.board.board)]  # Initialize with the initial board

    def find_next_states(self, current_board):
        next_states = []
        for action in range(12):
            pit_index = self.board.action2pit(action)
            if current_board[pit_index] != 0:
                copied_board = deepcopy(current_board)
                self.player.player_step(action, 1, 2)
                next_states.append(deepcopy(self.board.board))
        return next_states

    def game(self):
        print("START GAME ...")
        turn = 0
        while turn < self.max_turns and any(np.any(seeds != 0) for seeds in self.sub_states):
            next_level_states = []
            print(f"Turn {turn}:")
            for state in self.sub_states:
                print(f"Current board:\n{state}")
                next_states = self.find_next_states(state)
                print(f"Next states:\n{next_states}")
                self.board_states.extend(next_states)  # Accumulate all states
                next_level_states.extend(next_states)
            self.sub_states = next_level_states
            turn += 1
            if not self.sub_states:
                print("No more moves possible.")
                break

        print("Simulation completed.")

# Usage example
if __name__ == "__main__":
    simulator = GameSimulator()
    simulator.game()


START GAME ...
Turn 0:
Current board:
[[4 4 4 4 4 4]
 [4 4 4 4 4 4]]
Next states:
[array([[6, 1, 6, 1, 7, 2],
       [6, 6, 0, 1, 6, 6]]), array([[7, 2, 7, 2, 0, 2],
       [7, 7, 1, 1, 6, 6]]), array([[9, 1, 0, 1, 1, 0],
       [9, 1, 3, 3, 8, 8]]), array([[9, 1, 0, 1, 1, 0],
       [9, 1, 3, 3, 8, 8]]), array([[ 0,  0,  1,  2,  2,  1],
       [10,  2,  4,  4,  9,  9]]), array([[ 0,  0,  1,  2,  2,  1],
       [10,  2,  4,  4,  9,  9]]), array([[ 0,  1,  2,  3,  3,  2],
       [ 0,  3,  5,  5, 10, 10]]), array([[ 2,  0,  4,  5,  5,  4],
       [ 2,  0,  8,  1,  1, 12]]), array([[ 2,  1,  5,  6,  6,  5],
       [ 2,  0,  0,  2,  2, 13]]), array([[4, 3, 7, 8, 1, 8],
       [4, 2, 2, 0, 5, 0]]), array([[ 6,  0,  1, 10,  0, 10],
       [ 6,  4,  4,  0,  2,  1]]), array([[ 7,  1,  2, 11,  1,  0],
       [ 7,  5,  5,  1,  3,  1]])]
Turn 1:
Current board:
[[6 1 6 1 7 2]
 [6 6 0 1 6 6]]
Next states:
[array([[ 7,  1,  2, 11,  1,  0],
       [ 7,  5,  5,  1,  3,  1]]), array([[8, 2, 3, 1, 1, 1]

In [35]:
def find_unique_boards(board_states):
    unique_boards = set()
    # Convert each board array to a tuple and add to the set for uniqueness
    for board in board_states:
        board_tuple = tuple(map(tuple, board))  # Convert array to tuple of tuples
        unique_boards.add(board_tuple)
    
    # Convert tuples back to arrays if you need them in array form later
    unique_board_arrays = [np.array(board) for board in unique_boards]
    
    return unique_board_arrays


unique_boards = find_unique_boards(simulator.board_states)

unique_boards,  len(unique_boards)

([array([[0, 2, 1, 0, 0, 0],
         [3, 1, 1, 0, 0, 0]]),
  array([[5, 0, 2, 3, 0, 0],
         [1, 1, 0, 0, 0, 0]]),
  array([[3, 3, 6, 2, 7, 7],
         [1, 1, 0, 1, 1, 8]]),
  array([[1, 2, 1, 1, 0, 2],
         [0, 0, 3, 1, 0, 1]]),
  array([[4, 2, 0, 1, 2, 3],
         [0, 0, 0, 0, 0, 0]]),
  array([[1, 2, 1, 1, 1, 0],
         [0, 0, 0, 0, 0, 2]]),
  array([[ 6,  0,  1, 10,  0, 10],
         [ 6,  4,  4,  0,  2,  1]]),
  array([[4, 2, 1, 2, 3, 0],
         [0, 0, 0, 0, 0, 0]]),
  array([[4, 2, 0, 0, 0, 0],
         [0, 3, 2, 1, 0, 0]]),
  array([[0, 0, 1, 1, 1, 3],
         [8, 6, 3, 6, 3, 4]]),
  array([[0, 1, 2, 1, 3, 5],
         [0, 0, 5, 8, 5, 6]]),
  array([[9, 1, 0, 0, 0, 2],
         [7, 5, 2, 5, 2, 3]]),
  array([[2, 0, 0, 1, 1, 2],
         [2, 0, 0, 0, 0, 0]]),
  array([[1, 1, 2, 0, 2, 5],
         [0, 0, 0, 0, 0, 1]]),
  array([[4, 2, 0, 0, 1, 2],
         [0, 0, 0, 0, 0, 3]]),
  array([[1, 4, 5, 4, 3, 9],
         [3, 2, 1, 0, 0, 0]]),
  array([[4, 2, 0, 0, 0, 0],